## BaseLine

### Library

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import torch
import yaml
import warnings
from ultralytics import YOLO
from tqdm import tqdm
from pathlib import Path
warnings.filterwarnings('ignore')
import sys

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 환경에 따라 PROJECT_ROOT 설정
if IN_COLAB:
    PROJECT_ROOT = Path("/content/drive/MyDrive/likelion/PCB")
else:
    PROJECT_ROOT = Path.cwd().parent

# sys.path 추가 (import용)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# 작업 디렉토리 변경 (파일 접근용) - 중요!
os.chdir(PROJECT_ROOT)

print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"Current working directory: {os.getcwd()}")

### Train

In [ ]:
from src.training.train import train_model
from src.models.model_factory import ModelFactory
from src.utils.config import load_config

config_path = f"{PROJECT_ROOT}/configs/train_config.yaml"
config = load_config(config_path)
config

In [ ]:
train_model(config_path=config_path)

### Inference

In [ ]:
best_model_path = f"{PROJECT_ROOT}/outputs/yolov8s-roboflow_base/weights/best.pt"
test_path = f"{PROJECT_ROOT}/dataset/roboflow/test/"
save_dir = f"{PROJECT_ROOT}/runs"

print(f"Model Path: {best_model_path}")
print(f"Test Data Path: {test_path}")
print(f"Save Path: {save_dir}")

In [ ]:
from src.evaluation import Evaluator

# 평가 실행
evaluator = Evaluator(weights_path=best_model_path)

# Test 셋으로 평가 (mAP 계산)
evaluator = Evaluator(weights_path=best_model_path)
metrics = evaluator.evaluate(
    data_yaml='dataset/roboflow/data.yaml',
    split='test'
)

metrics_df = pd.DataFrame([metrics])
metrics_df.drop(columns=['per_class'], inplace=True)
metrics_df

In [ ]:
import random

def visualize_predict(model_path, test_images_dir, label_dir, num_samples=5, conf=0.25, figsize=(15, 5)):

    model = YOLO(model_path)

    # 테스트 이미지 수집
    test_images = list(Path(test_images_dir).glob('*.jpg'))
    test_images.extend(list(Path(test_images_dir).glob('*.png')))

    if len(test_images) == 0:
        print("No images found")
        return

    # 레이블이 있는 이미지만 필터링 (정상 이미지 제외)
    images_with_labels = []
    for img_path in test_images:
        label_path = Path(label_dir) / f"{img_path.stem}.txt"
        if label_path.exists():
            with open(label_path, 'r') as f:
                lines = f.readlines()
                if len(lines) > 0:
                    images_with_labels.append(img_path)

    print(f"Found {len(images_with_labels)} images with defects")

    if len(images_with_labels) == 0:
        print("No images with labels found")
        return

    samples = random.sample(images_with_labels, min(num_samples, len(images_with_labels)))

    for img_path in samples:
        # 예측 수행
        results = model(str(img_path), conf=conf, verbose=False)
        result = results[0]

        # 원본 이미지 로드
        img_orig = cv2.imread(str(img_path))
        img_orig = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
        h, w = img_orig.shape[:2]

        # Ground Truth 이미지
        img_gt = img_orig.copy()

        # 예측 이미지
        img_pred = img_orig.copy()

        # Ground Truth 레이블 읽기
        label_path = Path(label_dir) / f"{img_path.stem}.txt"
        gt_count = 0

        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls_id = int(parts[0])
                    cx, cy, bw, bh = map(float, parts[1:5])

                    # Normalized -> Pixel 좌표 변환
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)

                    cls_name = result.names[cls_id]

                    # Ground Truth는 녹색
                    color = (0, 255, 0)

                    # Ground Truth bbox 그리기
                    cv2.rectangle(img_gt, (x1, y1), (x2, y2), color, 4)
                    label = f"GT: {cls_name}"

                    # 레이블 배경
                    (label_w, label_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                    cv2.rectangle(img_gt, (x1, y1-label_h-10), (x1+label_w, y1), color, -1)
                    cv2.putText(img_gt, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                    gt_count += 1

        # 예측 바운딩 박스 그리기
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf_score = float(box.conf[0])
            cls_id = int(box.cls[0])
            cls_name = result.names[cls_id]

            # 예측은 빨간색
            color = (255, 0, 0)

            cv2.rectangle(img_pred, (x1, y1), (x2, y2), color, 4)
            label = f"{cls_name} {conf_score:.2f}"

            # 레이블 배경
            (label_w, label_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            cv2.rectangle(img_pred, (x1, y1-label_h-10), (x1+label_w, y1), color, -1)
            cv2.putText(img_pred, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # 시각화
        fig, axes = plt.subplots(1, 3, figsize=figsize)

        axes[0].imshow(img_orig)
        axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
        axes[0].axis('off')

        axes[1].imshow(img_gt)
        axes[1].set_title(f'Ground Truth ({gt_count} objects)', fontsize=14, fontweight='bold')
        axes[1].axis('off')

        axes[2].imshow(img_pred)
        axes[2].set_title(f'Predictions ({len(result.boxes)} detections)', fontsize=14, fontweight='bold')
        axes[2].axis('off')

        plt.suptitle(f'{img_path.name}', fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()

In [ ]:
visualize_predict(
    model_path=best_model_path,
    test_images_dir=f"{test_path}/images",
    label_dir=f"{test_path}/labels",
    num_samples=2,  # 5개 이미지 시각화
    conf=0.25
)

---

### EXP 1